<a href="https://colab.research.google.com/github/Kishan1082/Deep-Learning/blob/main/cifar10_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cifar10 dataset

In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, Activation
from keras.layers import BatchNormalization
from keras.utils import np_utils

from tensorflow.keras.applications import ResNet50

In [2]:
(X_train, Y_train),(X_test, y_test) = cifar10.load_data()
(X_train.shape, Y_train.shape),(X_test.shape, y_test.shape)

170508288/170498071 [==============================] - 6s 0us/step


(((50000, 32, 32, 3), (50000, 1)), ((10000, 32, 32, 3), (10000, 1)))

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, random_state = 4, test_size = 0.3)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((35000, 32, 32, 3), (15000, 32, 32, 3), (35000, 1), (15000, 1))

In [5]:
from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)
y_val = to_categorical(y_val,10)

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
      rotation_range = 2, 
      horizontal_flip = True,
      zoom_range = 1,
      )

test_generator = ImageDataGenerator(
      rotation_range = 2, 
      horizontal_flip = True,
      zoom_range = 1,
    )

val_generator = ImageDataGenerator(
      rotation_range = 2, 
      horizontal_flip = True,
      zoom_range = 1,
    )

In [7]:
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(X_test)

In [8]:
from keras.callbacks import ReduceLROnPlateau

In [9]:
lrr = ReduceLROnPlateau(monitor = "val_accuracy", factor = 0.01, patience=3, min_lr=1e-5)

In [10]:
model = Sequential()
model.add(ResNet50(weights = "imagenet", include_top = True))
model.add(Dense(10,activation = "softmax"))
model.summary()

102981632/102967424 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 dense (Dense)               (None, 10)                10010     
                                                                 
Total params: 25,646,722
Trainable params: 25,593,602
Non-trainable params: 53,120
_________________________________________________________________


In [11]:
from tensorflow.keras.optimizers import Adam
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = "accuracy")

In [12]:
model.fit_generator(
    train_generator.flow(x_train, y_train, batch_size = 128), 
    epochs = 5, 
    validation_data = val_generator.flow(x_val,y_val, batch_size = 128),
    callbacks = [lrr],
    verbose = 1 
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
274/274 [==============================] - 73s 199ms/step - loss: 2.2618 - accuracy: 0.1690 - val_loss: 2.3013 - val_accuracy: 0.1003 - lr: 0.0010
Epoch 2/5
274/274 [==============================] - 51s 188ms/step - loss: 2.1720 - accuracy: 0.1978 - val_loss: 2.2202 - val_accuracy: 0.1687 - lr: 0.0010
Epoch 3/5
274/274 [==============================] - 51s 188ms/step - loss: 2.1088 - accuracy: 0.2218 - val_loss: 2.1327 - val_accuracy: 0.2042 - lr: 0.0010
Epoch 4/5
274/274 [==============================] - 51s 188ms/step - loss: 2.0498 - accuracy: 0.2385 - val_loss: 2.1172 - val_accuracy: 0.2105 - lr: 0.0010
Epoch 5/5
274/274 [==============================] - 52s 188ms/step - loss: 1.9877 - accuracy: 0.2503 - val_loss: 1.9761 - val_accuracy: 0.2433 - lr: 0.0010
